In [1]:
import time

import zmq

ctx = zmq.Context()
# The REQ talks to Pupil remote and receives the session unique IPC SUB PORT
socket = ctx.socket(zmq.REQ)

ip = 'localhost'
port = 50020

socket.connect(f'tcp://{ip}:{port}')

# Request 'SUB_PORT' for reading data
socket.send_string('SUB_PORT')
sub_port = socket.recv_string()

# Request 'PUB_PORT' for writing data
socket.send_string('PUB_PORT')
pub_port = socket.recv_string()

socket.close()

In [2]:
import msgpack

def create_socket(ctx_c, ip_c, topics):
    sub = ctx_c.socket(zmq.SUB)
    sub.connect(f'tcp://{ip_c}:{sub_port}')
    for topic in topics:
        sub.subscribe(topic)
    return sub


In [3]:
import numpy as np

class HeadMovementDetector:
    def __init__(self, origin_pos, ptp_dist_max=0.05, error_tolerance=5):
        self.ptp_dist_max = ptp_dist_max
        self.error_tolerance = error_tolerance

        self.gaze_point_buffer = [origin_pos]

        self.finished = False

    def check_ptp_dist(self, gaze_point):
        ptp_dist = np.linalg.norm(np.array(self.gaze_point_buffer[-1]) - np.array(gaze_point))
        return ptp_dist < self.ptp_dist_max

    def update(self, gaze_point):
        if self.check_ptp_dist(gaze_point):
            self.gaze_point_buffer.append(gaze_point)
            return True
        else:
            if self.error_tolerance - 1 == 0:
                self.finished = True
                return False
            else:
                self.error_tolerance -= 1
                return True


In [6]:
import pygame
import sys
import time
import numpy as np
import msgpack

# Initialize pygame
pygame.init()

# Set the display to a square window
screen = pygame.display.set_mode((400, 400))
screen_width, screen_height = screen.get_size()

def update_screen(gaze_point, ost):
    gaze_point = np.array(gaze_point)
    gaze_point -= ost
    gaze_point = (
        screen_width - int(gaze_point[0] * screen_width),
        screen_height - int((1 - gaze_point[1]) * screen_height)
    )
    pygame.draw.circle(screen, RED, gaze_point, dot_radius // 2)
    pygame.display.flip()


# Colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# Red dot properties
dot_radius = 10

# Function to reset fixation
def reset_fixation():
    global fixation_pos, offset
    fixation_pos = None
    offset = None

# Main loop
running = True
fixation_pos = None
hmd = None
offset = None

while running:
    screen.fill(WHITE)  # Clear screen to white

    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
            running = False

        if event.type == pygame.KEYDOWN and event.key == pygame.K_r:
            reset_fixation()
            continue

    if fixation_pos is None:
        # Setup the socket for fixation data
        pupil_socket = create_socket(ctx, ip, ['fixation'])
        topic, payload = pupil_socket.recv_multipart()
        pupil_socket.close()

        # Get fixation data
        fixation_pos = list(msgpack.loads(payload)['norm_pos'])
        hmd = HeadMovementDetector(fixation_pos)
        print("Started head movement detection")

        offset = np.array(fixation_pos) - np.array([0.5, 0.5])  # Calculate offset
        fixation_pos = np.array(fixation_pos)
        fixation_pos -= offset  # Apply offset to fixation
        fixation_pos = (
            int(fixation_pos[0] * screen_width),
            int((1 - fixation_pos[1]) * screen_height)
        )
    else:
        pygame.draw.circle(screen, BLUE, fixation_pos, dot_radius)
        pygame.display.flip()
        # Setup the socket for gaze data
        pupil_socket = create_socket(ctx, ip, ['gaze.'])
        while not hmd.finished:
            topic, payload = pupil_socket.recv_multipart()
            message = msgpack.loads(payload)
            if message['confidence'] > 0.6:
                gaze_pos = list(message['norm_pos'])
                hmd.update(gaze_pos)  # Update head movement detector
                update_screen(gaze_pos, offset)  # Plot gaze point on screen

        pupil_socket.close()
        print("Head movement finished")
        print("Length of gaze point buffer:", len(hmd.gaze_point_buffer))

        pygame.draw.circle(screen, BLUE, fixation_pos, dot_radius)
        pygame.display.flip()

        # save a png of the screen by index number
        pygame.image.save(screen, f"./pics/head_movement_{time.time()}.png")

        reset_fixation()  # Reset after finishing head movement

# Quit pygame
pygame.quit()
sys.exit()

Started head movement detection
Head movement finished
Length of gaze point buffer: 313
Started head movement detection
Head movement finished
Length of gaze point buffer: 746
Started head movement detection
Head movement finished
Length of gaze point buffer: 308
Started head movement detection
Head movement finished
Length of gaze point buffer: 411
Started head movement detection
Head movement finished
Length of gaze point buffer: 550
Started head movement detection
Head movement finished
Length of gaze point buffer: 630
Started head movement detection
Head movement finished
Length of gaze point buffer: 14
Started head movement detection
Head movement finished
Length of gaze point buffer: 782
Started head movement detection
Head movement finished
Length of gaze point buffer: 509
Started head movement detection
Head movement finished
Length of gaze point buffer: 911
Started head movement detection
Head movement finished
Length of gaze point buffer: 135
Started head movement detection
H


KeyboardInterrupt

